In [1]:
import os
import shutil
import sys
from pathlib import Path

from rich import print
from rich.pretty import Pretty

abs_module_path = Path("./../../").resolve()
if (abs_module_path.exists()) and (str(abs_module_path) not in sys.path):
    sys.path.append(str(abs_module_path)) # add path to scan customized module

from modules.shared.config import load_config
from modules.shared.pathnavigator import PathNavigator
from modules.shared.utils import create_new_dir

In [ ]:
config = load_config("dataset.toml")
print(Pretty(config, expand_all=True))

In [ ]:
v1:bool
v2:bool

if config["dataset"].get("classif_strategy") == "": v1 = False
else: v1 = True

if config["dataset"].get("data_set") == "": v2 = False
else: v2 = True

assert not (v1 and v2), "Can't assigned both `classif_strategy` or `data_set` at a time"

if not v1:
    config["dataset"].pop("classif_strategy")
else:
    config["dataset"].pop("data_set")

config

In [ ]:
# Remove the parts of path you want to skip

# config["dataset"].pop("seed_dir")
# config["dataset"].pop("data")
# config["dataset"].pop("palmskin_result")
# config["dataset"].pop("base_size")
# config["dataset"].pop("classif_strategy")
# config["dataset"].pop("data_set")
# config["dataset"].pop("fish_dsname_root")
config["dataset"].pop("fish_dsname_sub")
# config["dataset"].pop("file_name")

config

In [ ]:
path_navigator = PathNavigator()
src_root: Path = path_navigator.dbpp.get_one_of_dbpp_roots("dataset_cropped_v3")
tmp_list: list = []
first_star: bool = False

for enum, (k, v) in enumerate(config["dataset"].items()):
    if ("*" in v) and (not first_star):
        first_star = True
        path = "/".join(tmp_list)
        src_root = src_root.joinpath(path)
        tmp_list = [v]
    else:
        tmp_list.append(v)

glob_pattern: str = "/".join(tmp_list)

src_root, glob_pattern

In [ ]:
img_paths = list(src_root.glob(glob_pattern))

len(img_paths), img_paths

In [ ]:
img_paths[0].parts

In [ ]:
dst_dir = Path(r"tmp") # Add directory for storing the collected images
create_new_dir(dst_dir)

for img_path in img_paths:
    
    # Modify to the file name you want
    save_name = img_path.stem
    save_name += f".{img_path.parts[9]}"
    save_name += f".{img_path.parts[10]}"
    save_name += img_path.suffix
    
    print(save_name)
    shutil.copy(img_path, dst_dir.joinpath(save_name))